In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
os.environ["TF_KERAS"] = '1'

tf.random.set_seed(1234)

folders_train = os.listdir('/data/home/zhaoyj/brain/mor_train')
folders_val = os.listdir('/data/home/zhaoyj/brain/mor_validation')




In [2]:
train_points = []
train_labels = []
count = 0
for folder in folders_train:
    folder_path = os.path.join('/data/home/zhaoyj/brain/mor_train',folder)
    files = os.listdir(folder_path)
    for file in files:
        file_path = os.path.join(folder_path,file)
        train_labels.append(count)
        tmp = np.loadtxt(file_path)
        tmp = tmp.tolist()
        train_points.append(tmp)        
    count += 1

In [3]:
val_points = []
val_labels = []
count = 0
for folder in folders_val:
    folder_path = os.path.join('/data/home/zhaoyj/brain/mor_validation',folder)
    files = os.listdir(folder_path)
    for file in files:
        file_path = os.path.join(folder_path,file)
        val_labels.append(count)
        tmp = np.loadtxt(file_path)
        tmp = tmp.tolist()
        val_points.append(tmp)        
    count += 1

In [4]:
train_points_array = np.array(train_points)

classes = set(train_labels)
class_dict = {c:np.identity(len(classes))[i,:] for i,c in enumerate(classes)}
train_labels_array = np.array(list(map(class_dict.get,train_labels)))


val_points_array = np.array(val_points)

classes2 = set(val_labels)
class_dict2 = {c:np.identity(len(classes2))[i,:] for i,c in enumerate(classes2)}
val_labels_array = np.array(list(map(class_dict2.get,val_labels)))

In [5]:
train_points_array.shape

(21, 8759, 3)

In [6]:
train_labels_array.shape

(21, 4)

In [7]:
val_points_array.shape

(20, 8759, 3)

In [8]:
val_labels_array.shape

(20, 4)

In [9]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

In [10]:
def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [11]:
NUM_POINTS = 2048
NUM_CLASSES = 4
inputs_x = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs_x, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=inputs_x, outputs=outputs, name="pointnet")
model.summary()

Model: "pointnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2048, 3)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 2048, 32)     128         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 2048, 32)     128         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 2048, 32)     0           batch_normalization[0][0]        
___________________________________________________________________________________________

In [12]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["acc"],
)

history = model.fit(train_points_array,train_labels_array,epochs=100,batch_size = 32,validation_data = (val_points_array,val_labels_array))

Epoch 1/100
1/1 [==============================] - 1s 826ms/step - loss: 3.5237 - acc: 0.2381 - val_loss: 48.0988 - val_acc: 0.2000
Epoch 2/100
1/1 [==============================] - 0s 194ms/step - loss: 3.0101 - acc: 0.2381 - val_loss: 4220.8193 - val_acc: 0.3000
Epoch 3/100
1/1 [==============================] - 0s 180ms/step - loss: 2.7080 - acc: 0.5238 - val_loss: 30261.1191 - val_acc: 0.1500
Epoch 4/100
1/1 [==============================] - 0s 180ms/step - loss: 2.6394 - acc: 0.5238 - val_loss: 17445.9590 - val_acc: 0.1500
Epoch 5/100
1/1 [==============================] - 0s 183ms/step - loss: 2.0311 - acc: 0.9048 - val_loss: 23582.1172 - val_acc: 0.1500
Epoch 6/100
1/1 [==============================] - 0s 179ms/step - loss: 2.0758 - acc: 0.8095 - val_loss: 2778.5725 - val_acc: 0.1500
Epoch 7/100
1/1 [==============================] - 0s 181ms/step - loss: 1.7996 - acc: 1.0000 - val_loss: 919.9232 - val_acc: 0.1500
Epoch 8/100
1/1 [==============================] - 0s 181ms/st

In [17]:
model.save_weights('./mor_classification_parameters.h5')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


NotImplementedError: <__main__.OrthogonalRegularizer object at 0x2ae472523250> does not implement get_config()

In [14]:
import pandas as pd
hist_df = pd.DataFrame(history.history)
hist_csv_file = './mor_classification_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)